In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

import numpy as np

24/05/15 16:42:01 WARN Utils: Your hostname, Yzeys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.192.255 instead (on interface en0)
24/05/15 16:42:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/15 16:42:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/15 16:42:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
def _bounds_coordinates(bin_bounds):

    lower_cdnts = [[low] for low in  bin_bounds[0][:-1]]
    upper_cdnts = [[high] for high in bin_bounds[0][1:]]
    
    # super stupid implementation, optimization needed
    for bound in bin_bounds[1:]:
        lower_tmp = []
        upper_tmp = []
        
        for bc in bound[:-1]:
            lower_tmp.extend([lc + [bc] for lc in lower_cdnts])
        lower_cdnts = lower_tmp
        
        for bc in bound[1:]:
            upper_tmp.extend([uc + [bc] for uc in upper_cdnts])
        upper_cdnts = upper_tmp
        
    return np.array(lower_cdnts), np.array(upper_cdnts)

def spatial_partition(dataset, par_each_dim, eps, withbuffer=True):
    """
    Partitions the given dataset into spatial bins based on the specified parameters.

    Args:
        dataset (numpy.ndarray): The input dataset to be partitioned.
        par_each_dim (tuple): A tuple specifying the number of partitions in each dimension.
        eps (float): A value representing the maximum distance between two samples.

    Returns:
        pyspark.rdd.RDD: An RDD containing the indexed data, where each element is a tuple
        of the partition ID and a list of point IDs belonging to that partition.
    """
    num_par = np.prod(par_each_dim)
    
    # partition each dimension
    bin_bounds = []
    for i in range(dataset.shape[1]):
        dim_bins = np.histogram_bin_edges(dataset[:, i], bins=par_each_dim[i])
        bin_bounds.append(dim_bins)
        
    lower_bounds, upper_bounds = _bounds_coordinates(bin_bounds)
    if withbuffer:
        lower_bounds -= eps
        upper_bounds += eps
        if np.min(upper_bounds-lower_bounds) < 2*eps:
            raise Warning('Partitions Overlap too much')

    # scatter points into bins with eps
    indexed_data = []
    # double loop to ensure border points could be given multiple partition ID
    for id_pts in range(len(dataset)):     # index of point in dataset
        for id_ptt in range(num_par):
            if not (dataset[id_pts] > lower_bounds[id_ptt]).all():
                continue
            if not (dataset[id_pts] < upper_bounds[id_ptt]).all():
                continue
            indexed_data.append([id_ptt, id_pts])
            
    res = sc.parallelize(indexed_data).groupByKey().map(lambda x: [x[0], list(x[1])])
    return res


In [1]:
from sklearn.datasets import make_blobs

# Generate random dataset with 3 dimensions
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 3  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)

In [4]:
# set parameters
min_pts = 18
eps = 2
dataset = X
b_dataset = sc.broadcast(dataset)
b_eps = sc.broadcast(eps)
b_min_pts = sc.broadcast(min_pts)
partition_each_dim = (2, 2, 2)

In [6]:
tp_par = partition_each_dim
num_par = np.prod(partition_each_dim)

# cut bins
bounds_dim = np.concatenate(([np.min(dataset, axis=0)], [np.max(dataset, axis=0)]), axis=0).T   # (d, 2)
bounds_dim

array([[-10.83328933,   4.5123324 ],
       [ -8.24949825,  10.41824487],
       [ -8.02990217,   8.01217509]])

In [93]:
bin_bounds = []
for i in range(len(partition_each_dim)):
    Lower_bound = bounds_dim[i][0] - abs(bounds_dim[i][0]) * 0.01
    Upper_bound = bounds_dim[i][-1] + abs(bounds_dim[i][-1]) * 0.01
    dim_bins = np.linspace(Lower_bound, Upper_bound, partition_each_dim[i]+1, endpoint=True)
    bin_bounds.append(dim_bins)
bin_bounds

[array([-10.94162222,  -3.19208325,   4.55745572]),
 array([-8.33199323,  1.09521704, 10.52242732]),
 array([-8.11020119, -0.00895218,  8.09229684])]

In [92]:
i= 1
np.linspace(*bounds_dim[i], partition_each_dim[i]+1, endpoint=True)

array([-8.24949825,  1.08437331, 10.41824487])

In [9]:
bin_bounds = []
for i in range(X.shape[1]):
    dim_bins = np.histogram_bin_edges(X[:, i], bins=partition_each_dim[i])
    bin_bounds.append(dim_bins)

bin_bounds

[array([-10.83328933,  -3.16047846,   4.5123324 ]),
 array([-8.24949825,  1.08437331, 10.41824487]),
 array([-8.02990217, -0.00886354,  8.01217509])]

In [22]:
import numpy as np

bin_bounds = [np.array([-10.83328933,  -3.16047846,   4.5123324 ]),
              np.array([-8.24949825,  1.08437331, 10.41824487]),
              np.array([-8.02990217, -0.00886354,  8.01217509])]

bounds = np.vstack(bin_bounds)
subspace_bounds = np.column_stack((np.min(bounds, axis=0), np.max(bounds, axis=0)))

print(subspace_bounds)

[[-10.83328933  -8.02990217]
 [ -3.16047846   1.08437331]
 [  4.5123324   10.41824487]]


In [10]:
import itertools

# Generate combinations of each dimension in bin_bounds
combinations = list(itertools.product(*bin_bounds))

# Print the combinations
for combination in combinations:
    print(combination)

(-10.83328932819366, -8.249498247867567, -8.029902170770304)
(-10.83328932819366, -8.249498247867567, -0.008863540457765495)
(-10.83328932819366, -8.249498247867567, 8.012175089854775)
(-10.83328932819366, 1.0843733094950245, -8.029902170770304)
(-10.83328932819366, 1.0843733094950245, -0.008863540457765495)
(-10.83328932819366, 1.0843733094950245, 8.012175089854775)
(-10.83328932819366, 10.418244866857616, -8.029902170770304)
(-10.83328932819366, 10.418244866857616, -0.008863540457765495)
(-10.83328932819366, 10.418244866857616, 8.012175089854775)
(-3.1604784639917973, -8.249498247867567, -8.029902170770304)
(-3.1604784639917973, -8.249498247867567, -0.008863540457765495)
(-3.1604784639917973, -8.249498247867567, 8.012175089854775)
(-3.1604784639917973, 1.0843733094950245, -8.029902170770304)
(-3.1604784639917973, 1.0843733094950245, -0.008863540457765495)
(-3.1604784639917973, 1.0843733094950245, 8.012175089854775)
(-3.1604784639917973, 10.418244866857616, -8.029902170770304)
(-3.160

In [14]:
len(combinations)

27

In [ ]:
lower_bounds, upper_bounds = _bounds_coordinates(bin_bounds)

In [16]:
lower_cdnts = [[low] for low in  bin_bounds[0][:-1]]
upper_cdnts = [[high] for high in bin_bounds[0][1:]]

# super stupid implementation, optimization needed
for bound in bin_bounds[1:]:
    lower_tmp = []
    upper_tmp = []
    
    for bc in bound[:-1]:
        lower_tmp.extend([lc + [bc] for lc in lower_cdnts])
    lower_cdnts = lower_tmp
    
    for bc in bound[1:]:
        upper_tmp.extend([uc + [bc] for uc in upper_cdnts])
    upper_cdnts = upper_tmp
    

In [23]:
lower_bounds, upper_bounds = np.array(lower_cdnts), np.array(upper_cdnts)

In [26]:
lower_bounds

array([[-1.08332893e+01, -8.24949825e+00, -8.02990217e+00],
       [-3.16047846e+00, -8.24949825e+00, -8.02990217e+00],
       [-1.08332893e+01,  1.08437331e+00, -8.02990217e+00],
       [-3.16047846e+00,  1.08437331e+00, -8.02990217e+00],
       [-1.08332893e+01, -8.24949825e+00, -8.86354046e-03],
       [-3.16047846e+00, -8.24949825e+00, -8.86354046e-03],
       [-1.08332893e+01,  1.08437331e+00, -8.86354046e-03],
       [-3.16047846e+00,  1.08437331e+00, -8.86354046e-03]])

In [27]:
upper_bounds

array([[-3.16047846e+00,  1.08437331e+00, -8.86354046e-03],
       [ 4.51233240e+00,  1.08437331e+00, -8.86354046e-03],
       [-3.16047846e+00,  1.04182449e+01, -8.86354046e-03],
       [ 4.51233240e+00,  1.04182449e+01, -8.86354046e-03],
       [-3.16047846e+00,  1.08437331e+00,  8.01217509e+00],
       [ 4.51233240e+00,  1.08437331e+00,  8.01217509e+00],
       [-3.16047846e+00,  1.04182449e+01,  8.01217509e+00],
       [ 4.51233240e+00,  1.04182449e+01,  8.01217509e+00]])

In [36]:
np.digitize(5, bin_bounds[0])

3

In [32]:
bin_bounds[0]

array([-10.83328933,  -3.16047846,   4.5123324 ])

In [33]:
dataset[0][0]

-2.936066417092473

In [94]:
indexed_data = []

for id_pts in range(len(dataset)):
    pos_list = []
    for i in range(dataset.shape[1]):
        pos = np.digitize(dataset[id_pts][i], bin_bounds[i]) - 1
        pos_list.append(pos)
    indexed_data.append([tuple(pos_list), id_pts])

In [95]:
def create_all_partition_id(partition_each_dim):
    """
    Create all possible partition IDs based on the specified number of partitions in each dimension.

    Args:
        partition_each_dim (tuple): A tuple specifying the number of partitions in each dimension.

    Returns:
        list: A list of all possible partition IDs.
    """
    return list(itertools.product(*[range(i) for i in partition_each_dim]))

partition_each_dim = (2, 2, 2)
par_ids = create_all_partition_id(partition_each_dim)


In [96]:
res = sc.parallelize(indexed_data).groupByKey().map(lambda x: [x[0], list(x[1])])

In [97]:
res.keys().collect()

[(1, 0, 0), (0, 1, 1), (1, 1, 1)]

In [86]:
res.filter(lambda x: x[0] == (1, 0, -1)).collect()

[[(1, 0, -1), [48]]]

In [84]:
dataset[48][2] == bin_bounds[2][0]

True

In [83]:
bin_bounds[2][0]

-8.029902170770304

In [88]:
res.filter(lambda x: x[0] == (1, -1, 0)).collect()

[[(1, -1, 0), [6]]]

In [89]:
dataset[6]

array([ 2.11903154, -8.24949825, -7.65189034])

In [90]:
for bound in bin_bounds:
    new_lower_bound = bound[0] - abs(bound[0]) * 0.01
    new_upper_bound = bound[-1] + abs(bound[-1]) * 0.01
    

[array([-10.83328933,  -3.16047846,   4.5123324 ]),
 array([-8.24949825,  1.08437331, 10.41824487]),
 array([-8.02990217, -0.00886354,  8.01217509])]